In [2]:
from IPython.core.display import HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
%config IPCompleter.use_jedi=False

# Tokenizing

In [4]:
!../mosesdecoder/scripts/tokenizer/tokenizer.perl -l en -threads 20 < ../data/raw/europarl-v7/fr/europarl-v7.fr-en.en > ../data/processed/europarl-v7/euro.tok.en
!../mosesdecoder/scripts/tokenizer/tokenizer.perl -l fr -threads 20 < ../data/raw/europarl-v7/fr/europarl-v7.fr-en.fr > ../data/processed/europarl-v7/euro.tok.fr

Tokenizer Version 1.1
Language: en
Number of threads: 20
Tokenizer Version 1.1
Language: fr
Number of threads: 20


# True casing

In [5]:
!../mosesdecoder/scripts/recaser/train-truecaser.perl --model ../models/truecaser/euro.model.en --corpus ../data/processed/europarl-v7/euro.tok.en
!../mosesdecoder/scripts/recaser/train-truecaser.perl --model ../models/truecaser/euro.model.fr --corpus ../data/processed/europarl-v7/euro.tok.fr

In [6]:
!../mosesdecoder/scripts/recaser/truecase.perl --model ../models/truecaser/euro.model.en < ../data/processed/europarl-v7/euro.tok.en > ../data/processed/europarl-v7/euro.true.en
!../mosesdecoder/scripts/recaser/truecase.perl --model ../models/truecaser/euro.model.fr < ../data/processed/europarl-v7/euro.tok.fr > ../data/processed/europarl-v7/euro.true.fr

# Cleaning

In [7]:
!../mosesdecoder/scripts/training/clean-corpus-n.perl ../data/processed/europarl-v7/euro.true fr en ../data/processed/europarl-v7/euro.clean 1 100

clean-corpus.perl: processing ../data/processed/europarl-v7/euro.true.fr & .en to ../data/processed/europarl-v7/euro.clean, cutoff 1-100, ratio 9
..........(100000)..........(200000)..........(300000)..........(400000)..........(500000)..........(600000)..........(700000)..........(800000)..........(900000)..........(1000000)..........(1100000)..........(1200000)..........(1300000)..........(1400000)..........(1500000)..........(1600000)..........(1700000)..........(1800000)..........(1900000)..........(2000000)
Input sentences: 2007723  Output sentences:  1991187


# Parsing

In [ ]:
java -jar ../berkeleyparser/BerkeleyParser-1.6.jar -gr ../berkeleyparser/eng_sm6.gr -inputFile ../data/processed/europarl-v7/euro.clean.en | ../mosesdecoder/scripts/training/wrappers/berkeleyparsed2mosesxml.perl > ../data/processed/europarl-v7/euro.syntax.en